In [7]:
import numpy as np
import tensorflow as tf
import keras

# For models
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.layers import  Dense, Input

import matplotlib.pyplot as plt 

In [4]:
# Initializing parameters 
k = 4
NUM_CHANNEL_USES = 4

sampling_factor = 4 # r
N_msg = 2*NUM_CHANNEL_USES*sampling_factor  # complex numbers converted into real
l = 6
N_seq =(2*l-1)*N_msg 
frame_size = 100*N_msg
q = 1   # strides = 1 (considered all values as real -> moving half of a complex number)

block_size = 32    # num of messages for frames we use, out of this, we use 1/4 as pilots and 3/4 as messages
n_blocks_train = 10**4  ################
n_blocks_val = 10**3

n_train = block_size * n_blocks_train
n_val   = block_size * n_blocks_val

In [8]:
def create_2d_array(arr, window_size, stride):
    num_windows = (len(arr) - window_size) // stride + 1
    shape = (num_windows, window_size)
    strides = (arr.strides[0] * stride, arr.strides[0])
    return np.lib.stride_tricks.as_strided(arr, shape=shape, strides=strides)

arr = np.array([1,2,3,4,5,6,7,8,9,10])
window_size = 7
stride = 1

result = create_2d_array(arr, window_size, stride)
print(result)

def create_2d_array_tf(arr, window_size, stride):
    num_windows = (arr.shape[0] - window_size) // stride + 1
    windows = []
    for i in range(num_windows):
        window = arr[i * stride:i * stride + window_size]
        windows.append(window)
    return tf.stack(windows)

arr = tf.constant([1,2,3,4,5,6,7,8,9,10], dtype=tf.int32)
window_size = 7
stride = 1

result = create_2d_array_tf(arr, window_size, stride)
print(result)

[[ 1  2  3  4  5  6  7]
 [ 2  3  4  5  6  7  8]
 [ 3  4  5  6  7  8  9]
 [ 4  5  6  7  8  9 10]]
tf.Tensor(
[[ 1  2  3  4  5  6  7]
 [ 2  3  4  5  6  7  8]
 [ 3  4  5  6  7  8  9]
 [ 4  5  6  7  8  9 10]], shape=(4, 7), dtype=int32)


In [25]:
import tensorflow as tf

def circular_shift(tensor):
    shape = tensor.shape
    rows = shape[0]
    for i in range(rows):
        shift_amount = i
        current_row = tensor[i, :]
        shifted_row = tf.roll(current_row, shift=shift_amount, axis=0)
        tensor = tf.tensor_scatter_nd_update(tensor, [[i]], tf.expand_dims(shifted_row, axis=0))
    return tensor

# Test the function with your example
input_tensor = tf.constant([[0,0,1,0],[0,1,0,0],[1,0,0,0],[0,0,0,1]], dtype=tf.int32)
result = circular_shift(input_tensor)
print(result)
print(tf.reduce_sum(result,0)/result.shape[1])

tf.Tensor(
[[0 0 1 0]
 [0 0 1 0]
 [0 0 1 0]
 [0 0 1 0]], shape=(4, 4), dtype=int32)
tf.Tensor([0. 0. 1. 0.], shape=(4,), dtype=float64)


In [6]:
# creating offset estimator
"""
input is the matrix with massage sequences as rows, 
different between two rows are q,
there are m = N-N_seq+1 rows for a frame
 """
class OffsetEstimator(keras.Model):
    def __init__(self):
        super().__init__()
        self.OE = Sequential([
            Input(shape=(N_seq,),name='OE_input'),
            Dense(256,input_shape=(N_seq,),name = 'dense_layer_1',activation='relu'),
            Dense(256,name = 'dense_layer_2',activation='relu'),
            Dense(256,name = 'dense_layer_3',activation='relu'),
            Dense(N_msg,name = 'dense_layer_4',activation='softmax'),
        ])
        self.loss_tracker = keras.metrics.Mean(name="loss")
        self.mae_metric = keras.metrics.MeanAbsoluteError(name="mae")
    def call(self,inputs):
        tau_vec_matrix = self.OE(inputs, training=False)
        tau_sum_vec = tf.reduce_sum(circular_shift(tau_vec_matrix),0)/result.shape[1]   # This vector has size of N_msg, and has the maximum mean value,                                                                         # corresponding to the shift
        r = np.argmax(tau_sum_vec) # (or use tf.math.argmax)
        i = r-1 -N_msg(np.fix((r-1)/(N_msg/2)))
        return i
    def compile(self, optimizer):
        super().compile()
        self.optimizer = optimizer
    def train_step(self,data):   # frame_offset is a index value [-8,7] if N_msg = 16. frame_offset is the label we use to train the model 
        inputs,frame_offset = data
        batch_size = inputs.shape[0]
        with tf.GradientTape() as tape:
            tau_vec_matrix = self.OE(inputs, training=True)
            tau_sum_vec = tf.reduce_sum(circular_shift(tau_vec_matrix),0)/result.shape[1]   # This vector has size of N_msg, and has the maximum mean value,                                                                         # corresponding to the shift
            r = np.argmax(tau_sum_vec) # (or use tf.math.argmax)
            i = r-1 -N_msg(np.fix((r-1)/(N_msg/2)))
            # compute the loss
            loss = keras.losses.mean_squared_error(i,frame_offset)
        # Compute gradient
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss,trainable_vars)
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        # matrics
        self.loss_tracker.update_state(loss)
        self.mae_metric.update_state(i, frame_offset)
        return {"loss": self.loss_tracker.result(), "mae": self.mae_metric.result()}
    def test_step(self,data):
        inputs,frame_offset = data
        tau_vec_matrix = self.OE(inputs, training=False)
        tau_sum_vec = tf.reduce_sum(circular_shift(tau_vec_matrix),0)/result.shape[1]   # This vector has size of N_msg, and has the maximum mean value,                                                                         # corresponding to the shift
        r = np.argmax(tau_sum_vec) # (or use tf.math.argmax)
        i = r-1 -N_msg(np.fix((r-1)/(N_msg/2)))
        # compute the loss
        loss = keras.losses.mean_squared_error(i,frame_offset)
        # matrics
        self.loss_tracker.update_state(loss)
        self.mae_metric.update_state(i, frame_offset)
        return {"loss": self.loss_tracker.result(), "mae": self.mae_metric.result()}
    @property
    def metrics(self):
        return [self.loss_tracker, self.mae_metric]
    
    
OE_model = OffsetEstimator()

In [ ]:
# Train OE

In [ ]:
def get_frame_offset(frame):
    seq_matrix = create_2d_array(frame,N_seq,q)
    i_b = OE_model(seq_matrix)   # Frame offset index value
    return i_b

